In [1]:
import numpy as np
import scipy.signal as signal
import pandas as pd
from wfdb import processing
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os

In [2]:
class ECGSignal():
    def __init__(self, ecg_signal, Fs=1000, lowcut=0.3, highcut=60, resample_fs=128):
        self.ecg_signal = ecg_signal
        self.Fs = Fs
        self.lowcut = lowcut
        self.highcut = highcut
        self.resample_fs = resample_fs

        self.filtered_ecg_signal = self.apply_filter_to_signal()
        self.resampled_ecg_signal = signal.resample(self.filtered_ecg_signal, (len(self.ecg_signal) // self.Fs) * self.resample_fs)

        self.conf = processing.XQRS.Conf(ref_period=0.4)
        self.xqrs = processing.XQRS(sig=ecg_signal, fs=Fs, conf=self.conf)
        self.xqrs.detect(learn=False)
        peaks = self.xqrs.qrs_inds / Fs
        self.RRI = np.diff(peaks)

    def get_rri(self):
        return self.RRI

    def butter_bandpass(self, order=4):
    # to gain paramaters of butterworth bandpass filter
        nyq = 0.5 * self.Fs
        low = self.lowcut / nyq
        high = self.highcut / nyq
        b, a = signal.butter(order, [low, high], btype='band')
        return b, a

    def apply_filter_to_signal(self, order=4):
    # apply butterworth bandpass filter to the signal
        b, a = self.butter_bandpass(order)
        filtered = signal.filtfilt(b, a, self.ecg_signal)
        return filtered